In [1]:
import numpy as np 
import pandas as pd
from collections import Counter
import lightgbm as lgb
import xgboost as xgb
import catboost as cbt
from pandas import DataFrame as DF
import gplearn
import tpot
import deap
from hyperopt import hp, fmin, rand, tpe, space_eval
from bayes_opt import BayesianOptimization
import time
import gc
from tqdm import tqdm
from sklearn.metrics import f1_score

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [27]:
# train = pd.read_hdf("../cache_data/multiclass_6724.hdf","train")
# valid = pd.read_hdf("../cache_data/multiclass_6724.hdf","valid")
# test = pd.read_hdf("../cache_data/multiclass_6724.hdf","test")

# train.shape,valid.shape,test.shape

In [210]:
train = pd.read_hdf("../cache_data/lgb_model.hdf","train").sort_index()
test = pd.read_hdf("../cache_data/lgb_model.hdf","test").sort_index()

In [211]:
num_classes = 12
label_name = 'click_mode'
train.rename(columns={num_classes:label_name,'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,'11':11},inplace=True)
test.rename(columns={num_classes:label_name,'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,'11':11},inplace=True)

In [212]:
print(train['click_mode'].value_counts())
train['click_mode'].isnull().sum()

2.0     466176
0.0     324817
7.0     286659
1.0     274329
9.0     205343
5.0     187332
3.0      83194
10.0     51678
4.0      41587
6.0      40162
11.0     31830
8.0       6893
Name: click_mode, dtype: int64


0

In [217]:
raw_train_score = f1_score(train[label_name],np.argmax(train[range(num_classes)].values,axis=1),average='weighted')
#raw_valid_score = f1_score(valid[label_name],np.argmax(valid[range(num_classes)].values,axis=1),average='weighted')

raw_train_score,#raw_valid_score

(0.6844480445894455,)

In [218]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0
        self.coef_arr = []
        self.val_score = []

    def _kappa_loss(self, coef, X, y):
        X_p = DF(np.copy(X))
#         X_v = valid[range(num_classes)]
        for i in range(len(coef)):
            X_p[i] *= coef[i]
#             X_v[i] *= coef[i]
        
        l1 = f1_score(y, np.argmax(X_p.values,axis=1), average="weighted")
#         l2 = f1_score(valid[label_name], np.argmax(X_v.values,axis=1), average="weighted")
        
        self.coef_arr.append(coef)
#         self.val_score.append(l2)
        
        print(list(coef.astype(np.float16)),' Train score = ',l1.astype(np.float32))#,' Valid score =',l2.astype(np.float16))
        return -l1

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y) 
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='Powell') 

    def predict(self, X, coef):
        X_p = DF(np.copy(X))
        for i in range(len(coef)):
            X_p[i] *= coef[i]
        return X_p

    def coefficients(self):
        return self.coef_['x']
    
#     def best_score(self):
#         return np.max(self.val_score)
    
#     def best_coef(self):
#         return self.coef_arr[np.argmax(self.val_score)]
# nelder-mead 
# Before =  0.6709307221687296
# After =  0.678495494646318
# GAP =  0.007564772477588355
# [0.92348139 0.71086394 0.65876489 1.55548237 1.73545927 0.62715862
#  1.45702358 0.59522381 1.18419033 0.57798275 0.90171286 1.05940425]

# Powell
# [1.36738918 1.03790291 0.77733245 2.33533351 3.25118434 0.82638933
#  1.9516095  0.95423545 1.52848168 0.88731548 1.19048762 1.12236339]

# Valid Counts =  Counter({2.0: 466176, 0.0: 324817, 7.0: 286659, 1.0: 274329, 9.0: 205343, 5.0: 187332, 3.0: 83194, 10.0: 51678, 4.0: 41587, 6.0: 40162, 11.0: 31830, 8.0: 6893})
# Predicted Counts =  Counter({2: 502354, 7: 312068, 1: 287486, 9: 265773, 0: 212699, 5: 200295, 10: 66450, 3: 46351, 11: 43035, 6: 31197, 4: 24955, 8: 7337})
# Before =  0.6709307221687296
# After =  0.6791416147396933
# GAP =  0.008210892570963635

In [219]:
cv_pred = train[range(num_classes)].values
y = train[label_name].values

In [208]:
# initial_coef = [1.1] * num_classes
# initial_coef = [1.26287028, 0.82937038, 0.5277721 , 1.64775053, 2.35868594,
#        0.60038381, 1.43501455, 0.74809782, 1.64015677, 0.75331299,
#        1.19730997, 0.91512994]
initial_coef = [1.19727551, 0.90333928, 0.68864398, 2.11965788, 2.66564214, 0.76702021,
                1.71968408, 0.81284811, 1.49193684, 0.784336,   1.0948337,  0.94776608] 

In [220]:
from functools import partial
import scipy as sp
from math import sqrt

optR = OptimizedRounder()
optR.fit(cv_pred, y)
best_score = optR.coefficients()
# (0.6770738132751616,)

[1.197, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6860828
[1.197, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6860828
[2.197, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68359727


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[-0.4207, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.5884497
[1.197, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6860828
[0.579, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6858426
[1.579, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6857964
[1.059, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6860492
[1.343, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6860647
[1.223, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6860782
[1.145, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68609595
[1.111, 0.9033, 0.6885, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score = 

[1.186, 0.9404, 0.67, 2.111, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68613195
[1.186, 0.9404, 0.67, 2.14, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6861269
[1.186, 0.9404, 0.67, 2.123, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6861358
[1.186, 0.9404, 0.67, 2.121, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68613607
[1.186, 0.9404, 0.67, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68613607
[1.186, 0.9404, 0.67, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68613607
[1.186, 0.9404, 0.67, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68613607
[1.186, 0.9404, 0.67, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68613607
[1.186, 0.9404, 0.67, 2.12, 2.666, 0.767, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68613607


[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 0.1016, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.67871886
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.72, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68617934
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.102, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6861222
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 2.102, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68614405
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.661, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68616563
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.848, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6861799
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.945, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.68616354
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.785, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6861799
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.815, 0.813, 1.492, 0.784, 1.095, 0.9478]  Train score =  0.6861

...

[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.9478]  Train score =  0.686201
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.9478]  Train score =  0.686201
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.9478]  Train score =  0.686201
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.9478]  Train score =  0.686201
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.9478]  Train score =  0.686201
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.9478]  Train score =  0.686201
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.9478]  Train score =  0.686201
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.9478]  Train score =  0.686201
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 1.948]  Train score =  0.6861406
[

[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6862161
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6862161
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6862161
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6862161
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6862161
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6862161
[1.186, 0.9404, 1.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6860705
[1.186, 0.9404, -0.9478, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.45395926
[1.186, 0.9404, 0.67, 2.12, 3.594, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.68621

[1.186, 0.9404, 0.669, 2.121, 3.592, -0.9414, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.59956115
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.677, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6862169
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.0588, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.59956115
[1.186, 0.9404, 0.669, 2.121, 3.592, 1.059, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6861712
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.867, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6861714
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.4407, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.68573314
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.587, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.68612826
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.7495, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6861705
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.6426, 1.825, 0.813, 1.489, 0.9517, 1.095, 0.712]

[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, -0.1285, 0.9517, 1.095, 0.712]  Train score =  0.68467283
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, 1.489, 0.9517, 1.095, 0.712]  Train score =  0.6862171
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, 0.8716, 0.9517, 1.095, 0.712]  Train score =  0.6861331
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, 1.871, 0.9517, 1.095, 0.712]  Train score =  0.6862034
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, 1.576, 0.9517, 1.095, 0.712]  Train score =  0.68621397
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, 1.254, 0.9517, 1.095, 0.712]  Train score =  0.68621093
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, 1.438, 0.9517, 1.095, 0.712]  Train score =  0.6862193
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, 1.368, 0.9517, 1.095, 0.712]  Train score =  0.6862132
[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, 1.412, 0.9517, 1.095, 0

[1.186, 0.9404, 0.669, 2.121, 3.592, 0.676, 1.826, 0.8125, 1.438, 0.9487, 1.095, 0.769]  Train score =  0.68621683


In [221]:
best_coef = optR.coefficients()
best_coef#,best_score

array([1.18546017, 0.9404844 , 0.66918642, 2.12193384, 3.5909713 ,
       0.67592458, 1.82587149, 0.8126954 , 1.43834878, 0.94863208,
       1.09424807, 0.76961194])

In [223]:
print("Global Best")

pred_test_y_k = optR.predict(cv_pred, best_coef)
print(best_coef)
print("\nValid Counts = ", Counter(y))
print("Predicted Counts = ", Counter(np.argmax(pred_test_y_k.values,axis=1)))
acc1 = f1_score(y,np.argmax(cv_pred,axis=1),average="weighted")
acc2 = f1_score(y,np.argmax(pred_test_y_k.values,axis=1),average="weighted")
print("Train Before = ",acc1)
print("Train After = ",acc2)
print("Train GAP = ",acc2-acc1)

test_pred = optR.predict(test[range(num_classes)], best_coef)
test_pred = np.argmax(test_pred.values,axis=1)

Global Best
[1.18546017 0.9404844  0.66918642 2.12193384 3.5909713  0.67592458
 1.82587149 0.8126954  1.43834878 0.94863208 1.09424807 0.76961194]

Valid Counts =  Counter({2.0: 466176, 0.0: 324817, 7.0: 286659, 1.0: 274329, 9.0: 205343, 5.0: 187332, 3.0: 83194, 10.0: 51678, 4.0: 41587, 6.0: 40162, 11.0: 31830, 8.0: 6893})
Predicted Counts =  Counter({2: 500724, 7: 305544, 1: 281431, 9: 263300, 0: 225816, 5: 200693, 10: 67208, 3: 51317, 11: 39570, 6: 31095, 4: 24924, 8: 8378})
Train Before =  0.6844480445894455
Train After =  0.6862247141859121
Train GAP =  0.0017766695964666068


In [224]:
test_1 = pd.read_hdf("../cache_data/stack/677000_685361_post_powell.hdf","train")

In [225]:
for k in range(12):
    print(k,np.corrcoef(test_1[k],DF(pred_test_y_k)[k])[0][1])

0 0.82487611500043
1 0.9293025566523039
2 0.9757924986856176
3 0.7855396002527191
4 0.7485753840848436
5 0.9701076329672038
6 0.8313254092248018
7 0.9543064861065507
8 0.8454444546792
9 0.9619451043163072
10 0.9291262511372624
11 0.9462778960785544


In [226]:
np.corrcoef(np.argmax(test_1[range(num_classes)].values,axis=1),np.argmax(pred_test_y_k.values,axis=1))

array([[1.       , 0.9648866],
       [0.9648866, 1.       ]])

In [227]:
train_post = DF(pred_test_y_k).copy()
train_post.index = train.index
test_post = DF(optR.predict(test[range(num_classes)], best_coef)).copy()
test_post.index = test.index
train_post[12] = train['click_mode'].values
test_post[12] = np.nan

train_post.to_hdf("../cache_data/stack/{}_{}_post_powell.hdf".format(acc1,acc2),"train")
test_post.to_hdf("../cache_data/stack/{}_{}_post_powell.hdf".format(acc1,acc2),"test")

In [56]:
test['max_proba'] = test[range(num_classes)].values.max(axis=1)
test['max_proba_to_pred'] = test[range(num_classes)].values.argmax(axis=1)

In [229]:
test_pred = optR.predict(test[range(num_classes)], best_coef)
test_pred = np.argmax(test_pred.values,axis=1)

In [230]:
from collections import Counter
Counter(test_pred)

Counter({2: 89578,
         1: 46075,
         7: 55577,
         3: 6108,
         5: 33152,
         4: 3414,
         9: 29132,
         10: 8659,
         0: 24190,
         6: 2972,
         8: 1460,
         11: 4599})

In [148]:
test['label'] = (test['click_mode']==test['max_proba_to_pred']).astype(int)

In [231]:
find['label'].value_counts()

NameError: name 'find' is not defined

In [155]:
find[find['max_proba_to_pred']==2]#['label'].value_counts()

,0,1,2,3,4,5,6,7,8,9,10,11,click_mode,max_proba,max_proba_to_pred,label
sid,,,,,,,,,,,,,,,,
841034,0.018634,0.000835,0.944897,0.008138,0.004423,0.000186,0.002434,0.002127,0.000021,0.018122,0.000118,0.000066,2.0,0.944897,2,1
841042,0.018853,0.000363,0.960628,0.013782,0.005126,0.000158,0.000058,0.000555,0.000029,0.000250,0.000143,0.000055,0.0,0.960628,2,0
841043,0.017813,0.000321,0.971290,0.005021,0.004320,0.000136,0.000048,0.000571,0.000033,0.000217,0.000157,0.000073,2.0,0.971290,2,1
841053,0.024359,0.000807,0.961608,0.005176,0.003302,0.000281,0.003301,0.000685,0.000020,0.000290,0.000103,0.000067,2.0,0.961608,2,1
841134,0.016017,0.000339,0.969569,0.007396,0.005396,0.000145,0.000200,0.000524,0.000025,0.000222,0.000118,0.000048,2.0,0.969569,2,1
841191,0.022472,0.000762,0.954330,0.013668,0.004284,0.000195,0.003059,0.000758,0.000027,0.000278,0.000105,0.000062,2.0,0.954330,2,1
841201,0.014240,0.000286,0.974631,0.006016,0.003757,0.000120,0.000072,0.000483,0.000021,0.000185,0.000147,0.000042,2.0,0.974631,2,1
841214,0.010201,0.000399,0.957273,0.003267,0.002797,0.000152,0.000055,0.006174,0.000019,0.019223,0.000389,0.000052,2.0,0.957273,2,1
841215,0.009178,0.000298,0.982429,0.004196,0.002810,0.000122,0.000043,0.000510,0.000028,0.000193,0.000129,0.000063,2.0,0.982429,2,1


In [232]:
find = test[test['max_proba']>=0.9]
find['max_proba_to_pred'].value_counts()

KeyError: 'max_proba'

In [234]:
submit = pd.DataFrame()
submit['sid'] = test.index
submit['pred'] = test_pred
# submit['click_mode'] = test['click_mode'].values
submit.head(10)

,sid,pred
0,2000000,2
1,2000001,1
2,2000002,7
3,2000003,2
4,2000004,3
5,2000005,1
6,2000006,5
7,2000007,2
8,2000008,1
9,2000009,3


In [236]:
# f1_score(submit['click_mode'],submit['pred'],average='weighted')

In [237]:
submit[['sid','pred']].to_csv("../submit/6862lgb_post.csv",index=False,header=False)

In [106]:
a1 =[0.92348139, 0.71086394, 0.65876489, 1.55548237, 1.73545927, 0.62715862,
 1.45702358, 0.59522381, 1.18419033, 0.57798275, 0.90171286, 1.05940425,]

In [109]:
a3 = [1.17, 0.89, 0.65, 1.97, 2.78, 0.75, 1.67, 0.82, 1.35, 0.7, 1.02, 1.0]

In [107]:
a2=[1.36738918, 1.03790291, 0.77733245, 2.33533351, 3.25118434, 0.82638933,
 1.9516095,  0.95423545, 1.52848168, 0.88731548, 1.19048762, 1.12236339,]

In [115]:
a4=[1.2717398,  0.97906011, 0.7051182,  2.13323559, 2.09161593, 0.99742707,
 2.09484136, 0.84893617, 1.97479275, 0.77382245, 1.04814823, 1.01177361,]

In [108]:
np.corrcoef(a1,a2)

array([[1.        , 0.93978317],
       [0.93978317, 1.        ]])

In [110]:
np.corrcoef(a1,a3)

array([[1.        , 0.94494207],
       [0.94494207, 1.        ]])

In [111]:
np.corrcoef(a2,a3)

array([[1.        , 0.99890716],
       [0.99890716, 1.        ]])

In [116]:
np.corrcoef(a1,a4)

array([[1.        , 0.93074962],
       [0.93074962, 1.        ]])

In [117]:
np.corrcoef(a2,a4)

array([[1.        , 0.86581154],
       [0.86581154, 1.        ]])

In [118]:
np.corrcoef(a3,a4)

array([[1.        , 0.87289529],
       [0.87289529, 1.        ]])

In [122]:
j1 = optR.predict(cv_pred, a1)
j2 = optR.predict(cv_pred, a2)
j3 = optR.predict(cv_pred, a3)
j4 = optR.predict(cv_pred, a4)


In [125]:
np.corrcoef(np.argmax(j1.values,axis=1),np.argmax(j2.values,axis=1))

array([[1.        , 0.97284843],
       [0.97284843, 1.        ]])

In [127]:
np.corrcoef(np.argmax(j1.values,axis=1),np.argmax(j4.values,axis=1))

array([[1.        , 0.97295046],
       [0.97295046, 1.        ]])

In [128]:
np.corrcoef(np.argmax(j2.values,axis=1),np.argmax(j4.values,axis=1))

array([[1.        , 0.97884222],
       [0.97884222, 1.        ]])

In [133]:
submit = []
jj1 = np.argmax(j1.values,axis=1)
jj2 = np.argmax(j2.values,axis=1)
jj4 = np.argmax(j4.values,axis=1)

for i in tqdm(range(len(jj2))):
    if jj1[i]==jj2[i]:
        submit.append(jj1[i])
    elif jj1[i]==jj4[i]:
        submit.append(jj1[i])
    elif jj2[i]==jj4[i]:
        submit.append(jj2[i])
    else:
        submit.append(jj2[i])


100%|██████████| 2000000/2000000 [00:01<00:00, 1341329.35it/s]

In [134]:
f1_score(y,submit,average='weighted')

0.6788562137172918